In [1]:
# import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

In [2]:
# function to calculate y (target used in SWR)


def gen_y(today_close, yester_close, action_level):
    bound = 0.05 # percent to do full buy or full sell
    percent = (today_close - yester_close) / yester_close
    print(percent)
    ret = (percent + bound)/(bound*2)
    #print(ret)
    if ret > 1:
        return 1
    elif ret < 0:
        return -1
    else:
        return ret


In [3]:
# original features

def feature_5_conti(k_today, d_today, k_yestd, d_yestd, action, action_level):
    inter = (k_yestd+d_yestd+k_today+d_today)/4
    sinter = inter/100 # scaled inter
    if((k_yestd > d_yestd and k_today < d_today and inter > 50) or \
       (k_yestd < d_yestd and k_today > d_today and inter < 50)):
        # saction= sell all:1, sell:1~0.5, hold:0.5, buy:0.5~0, buy all:0
        # the diff btw saction & sinter(i.e. inter/100) : smaller is better(then return larger value)
        
        if(action <= action_level):
            saction = (action_level - action)/(2*action_level) # scaled action
        else:
            saction = action/(2*action_level)
            
        return 1-abs(sinter-saction)
        
    else: # best to hold
        if(action <= action_level): # hold
            return (action_level - action)/float(action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(action_level)
    
# from feature_6_my  
def feature_5_dis(k_today, d_today, k_yestd, d_yestd, action, action_level):
    inter = (k_yestd+d_yestd+k_today+d_today)/4
    if(k_yestd < d_yestd and k_today > d_today and inter < 30): # buy
        if(action <= action_level):
            return (action + action_level)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(2*action_level)
       
    elif(k_yestd > d_yestd and k_today < d_today and inter > 70): # sell
        if(action <= action_level):
            return (action_level - action)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (action)/float(2*action_level)
    else:
        if(action <= action_level): # hold
            return (action_level - action)/float(action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(action_level)
    
def psy(price_list):
    n = 0
    for i in range(1, len(price_list)):
        if price_list[i] > price_list[i-1]:
            n += 1
    return (n/float(len(price_list) - 1))*100


def feature_6_original(price_list, action, action_level):
    psy_num = psy(price_list)
    if(psy_num <= 25): # market oversell we can buy
        if(action <= action_level):
            return (action + action_level + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action - action_level)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)
    if(25 < psy_num < 75):#hold to observe situation
        if(action <= action_level):
            return 2*(action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return 2*(2*action_level - action + 1)/float(2*action_level + 1)
        else:
            return 1
    if(psy_num >= 75):#market overbuy we can sell
        if(action <= action_level):
            return (action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action + 1)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)
        
        
def feature_6_dis(price_list, action, action_level):
    psy_num = psy(price_list)
    if(psy_num <= 25): # market oversell we can buy
        if(action <= action_level):
            return (action + action_level)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(2*action_level)
        
    if(25 < psy_num < 75):#hold to observe situation
        if(action <= action_level):
            return (action_level - action)/float(action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(action_level)
        
    if(psy_num >= 75):#market overbuy we can sell
        if(action <= action_level):
            return (action_level - action)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (action)/float(2*action_level)
        
def feature_6_conti(price_list, action, action_level):
    psy_num = psy(price_list)
    spsy_num = psy_num/100

    if(action <= action_level):
        saction = (action_level - action)/(2*action_level) # scaled action
    else:
        saction = action/(2*action_level)
            
    return 1-abs(spsy_num-saction)


def feature_7(price_list, state, action, action_level):
    if(action > 2*action_level) or action < 0:
        sys.exit("error, action undefined!")
        
    price_avg = sum(price_list) / float(len(price_list))
    #print('price_avg = ', price_avg)
    
    #stock price is higher than avg_price means the stock is going up
    if price_avg <= state['StockPrice']:
    #if price_avg <= math.floor(state['StockPrice']):
        price_trend = 1
    else:
        price_trend = 0
    #print('price_trend= ', price_trend)
    
    #stock is going up, we should buy
    if price_trend == 1:
        if(action <= action_level):
            #print('in1')
            return (action + action_level)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            #print('in2')
            return (2*action_level - action)/float(2*action_level)
    #stock is going down, sell
    elif price_trend == 0:
        if(action <= action_level):
            #print('in3')
            return (action_level - action)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            #print('in4')
            return (action)/float(2*action_level)
        
def list_rev_count(inlist):
    ret = 0
    for i in range(2, len(inlist)):
        if (inlist[i] - inlist[i-1]) * (inlist[i-1] - inlist[i-2]) < 0:
            ret = ret + 1
    return ret

def feature_8_vari(price_list, action, action_level):
    if(action > 2*action_level) or action < 0:
        sys.exit("error, action undefined!")
    
    revcount = list_rev_count(price_list)
    
    if revcount > len(price_list) / 3:  # best to hold
        if(action <= action_level): 
            return (action_level - action)/float(action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(action_level)
    else:
        return 0
    
def open_price_trend(state, last_state):
    # op higher than yesterday's high -> trend is up
    op_trend = 0.5
    if(state['OpenPrice'] > last_state['HighPrice']):
        op_trend += 0.2
        # up over 5% -> trend is up
        if((state['OpenPrice'] - last_state['StockPrice'])/float(last_state['StockPrice']) > 0.05):
            op_trend += 0.3
        
    # op is lower than yesterday's low -> trend is down
    if(state['OpenPrice'] < last_state['LowPrice']):
        op_trend -= 0.2
        # down over 5% -> trend is down
        if((state['OpenPrice'] - last_state['StockPrice'])/float(last_state['StockPrice']) < -0.05):
            op_trend -= 0.3
    return op_trend

def feature_9_open(version, optrend_1, optrend_2, optrend_3, optrend_4, optrend_5, action, action_level):
    if version == 1:
        optrend = optrend_1
    elif version == 2:
        trend_3day = float(optrend_1 + optrend2 + optrend_3)/float(3)
        trend = trend_3day*(0.33) + optrend_1*(0.67)
    elif version == 3:
        trend_5day = float(optrend_1+optrend_2+optrend_3+optrend_4+optrend_5)/float(5)
        trend_3day = float(optrend_1+optrend_2+optrend_3)/float(3)
        trend = trend_5day*(0.2)+trend_3day*(0.3)+optrend_1*(0.5)
    else:
        sys.exit("error, version undefined! feature_9_open")
    
    #suggest to buy
    if(trend > 0.5):
        if(action <= action_level):
            return (action + action_level + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action - action_level)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)
    
    #keep observe
    if(trend == 0.5):
        if(action <= action_level):
            return 2*(action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return 2*(2*action_level - action + 1)/float(2*action_level + 1)
        else:
            return 1
    #suggest to sell
    if(trend < 0.5):
        if(action <= action_level):
            return (action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action + 1)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)

In [4]:
# gen X (i.e. best action) functions (based on feature functions)
# no 'action' parameter comparing with the corresponding feature functions

def to_one_one(action, action_level):
    if action > action_level:
        return (-1)*(action - action_level) / float(action_level)
    else:
        return action / float(action_level)

def gen_action_5_conti(k_today, d_today, k_yestd, d_yestd, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_5_conti(k_today, d_today, k_yestd, d_yestd, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_5_dis(k_today, d_today, k_yestd, d_yestd, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_5_dis(k_today, d_today, k_yestd, d_yestd, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_6_original(price_list, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_6_original(price_list, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_6_dis(price_list, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_6_dis(price_list, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_6_conti(price_list, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_6_conti(price_list, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_7(price_list, state, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_7(price_list, state, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_8_vari(price_list, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_8_vari(price_list, action, action_level)
    return to_one_one(np.argmax(results), action_level)


def gen_action_9_open(version, optrend_1, optrend_2, optrend_3, optrend_4, optrend_5, action_level):
    results = np.zeros(2*action_level + 1) # to store the feature output of each action
    for action in range(0, 2*action_level+1):
        results[action] = feature_9_open(version, optrend_1, optrend_2, optrend_3, optrend_4, optrend_5, action, action_level)
    return to_one_one(np.argmax(results), action_level)
    

In [5]:
# main function

from random import randint
import math
import stockstats

inputFolderFormat = "./input/{0}" 
inputCsvList = ['QCOM.csv']


action_level = 3

#of candidate feature : 7
#feature_5 = feature_5_conti
#feature_6 = feature_6_dis
f9version = 3


for resource_data in inputCsvList:
    input_csv = inputFolderFormat.format(resource_data)
    data = pd.read_csv(input_csv)
    close = data['Close']
    close = close.fillna(method='ffill')
    close = close.tolist()
    openn = data['Open']
    openn = openn.fillna(method='ffill')
    openn = openn.tolist()
    high = data['High']
    high = high.fillna(method='ffill')
    high = high.tolist()
    low = data['Low']
    low = low.fillna(method='ffill')
    low = low.tolist()

    # preparing data for feature_5
    StockStat = stockstats.StockDataFrame.retype(data)
    StockStat[['kdjk','kdjd','kdjj']]
    kdjk = StockStat['kdjk'].tolist()
    kdjd = StockStat['kdjd'].tolist()

    # initial state
    state = {'NumStock': 0, 'StockPrice': close[0],\
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_state = {'NumStock': 0, 'StockPrice': close[0],\
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_1_state = {'NumStock': 0, 'StockPrice': close[0],\
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_2_state = {'NumStock': 0, 'StockPrice': close[0], \
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_3_state = {'NumStock': 0, 'StockPrice': close[0], \
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_4_state = {'NumStock': 0, 'StockPrice': close[0], \
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_5_state = {'NumStock': 0, 'StockPrice': close[0], \
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}
    last_6_state = {'NumStock': 0, 'StockPrice': close[0], \
                     'OpenPrice': openn[0], 'HighPrice': high[0], 'LowPrice': low[0]}

    fx_num = 5
    
    
    # create a dataframe for X
    X = pd.DataFrame(0.0, index=np.arange(len(data)), columns=["5_conti", "5_dis", "6_original", "6_dis", "6_conti", 
                                                             "7", "8_vari", "9_open"])
    # create a series for y
    y = pd.Series(0.0, index=np.arange(len(data)))

    for point in range(fx_num, len(data)):
        
        #print('-----------------------------------', point)

        # new price revealed!
        state['NumStock'] = last_state['NumStock']
        state['StockPrice'] = close[point]
        state['OpenPrice'] = openn[point]
        state['HighPrice'] = high[point]
        state['LowPrice'] = low[point]


        # prepare for the arguments passed later
        #last_fx_list = close[point-fx_num:point]
        price_list = close[point-fx_num+1:point+1]

        # prepare for the arguments passed later
        last_1_k = kdjk[point-1]  
        last_1_d = kdjd[point-1] 
        last_2_k = kdjk[point-2] 
        last_2_d = kdjd[point-2] 
        now_k = kdjk[point]
        now_d = kdjd[point]


        # prepare for feature_9_open 
        optrend_1 = open_price_trend(state, last_1_state)
        optrend_2 = open_price_trend(last_1_state,last_2_state)
        optrend_3 = open_price_trend(last_2_state,last_3_state)
        optrend_4 = open_price_trend(last_3_state,last_4_state)
        optrend_5 = open_price_trend(last_4_state,last_5_state)
        optrend_6 = open_price_trend(last_5_state,last_6_state)
        
        
        
        # do x assignments
        
        X.at[X.index[point], '5_conti'] = gen_action_5_conti(now_k, now_d, last_1_k, last_1_d, action_level)

        X.at[X.index[point], '5_dis'] = gen_action_5_dis(now_k, now_d, last_1_k, last_1_d, action_level)

        X.at[X.index[point], '6_original'] = gen_action_6_original(price_list, action_level)

        X.at[X.index[point], '6_dis'] = gen_action_6_dis(price_list, action_level)

        X.at[X.index[point], '6_conti'] = gen_action_6_conti(price_list, action_level)

        X.at[X.index[point], '7'] = gen_action_7(price_list, state, action_level)

        X.at[X.index[point], '8_vari'] = gen_action_8_vari(price_list, action_level)

        X.at[X.index[point], '9_open'] = gen_action_9_open(f9version, optrend_1, optrend_2, optrend_3, optrend_4, optrend_5, action_level)


        y[point-1] = gen_y(state['StockPrice'], last_1_state['StockPrice'], action_level)


        last_1_state = state.copy()
        last_2_state = last_1_state.copy()
        last_3_state = last_2_state.copy()
        last_4_state = last_3_state.copy()
        last_5_state = last_4_state.copy()
        last_6_state = last_5_state.copy()
    
    


-0.03208394447154178
0.0024783612911512673
0.015760151177995247
-0.0059324613913131026
0.008722264995325527
-0.0037924758432183207
0.014466347326521991
0.0009005853692322688
0.0038992501499700064
-0.0140424555111308
0.003787878787878788
-0.0042264000000000555
-0.0024253902921723956
-0.009876827530066062
0.010742740285896793
0.0015183419143889683
0.020163766224435622
-0.0038639024540956693
-0.005221527752074065
-0.01784646070785853
0.004886242323598684
-0.024008434098417644
-0.009185800598514692
0.004556898248851382
0.01016740184576893
0.011923133727992016
0.009946473142967819
-0.053939409090909086
-0.01473412899958562
0.0014629551365409766
-0.0001623600038572414
0.005194805363467711
0.01566539133286142
0.01399270176487055
0.0014113376195703446
0.0017225179811694558
0.0023448803760374106
-0.00015598876375598527
0.008267009697909788
0.002784668978723067
0.007713668620796052
0.0033680496019597456
0.0015257552406811735
0.014777605343960986
-0.007656552955163558
-0.0051436612751485904
-0.00

0.016761834283259572
-0.01594204681264408
0.01859355670103093
-0.012109198911650207
-0.006403201609952376
0.01031113956341117
0.006014251977660877
-0.003985525290117358
-0.009639850854856248
-0.0011019650853634389
-0.00827357985426688
-0.007044827716811795
-0.005414506993754474
-0.0009385958325511484
-0.012777151206742736
-0.0020936619319821483
0.024794964714857853
-0.010236367020286566
0.035539676570139164
0.02106410023606313
0.001600569091232499
0.005149165482954612
0.01854794173206237
0.0020811481096080467
0.00017310488357757107
-0.004845180743291694
0.019474926777079772
0.010745300857149838
0.010968647012104198
-0.013520313804039346
-0.0016920135936383512
-0.0005084576271185857
0.02475833432663493
0.0049644216448783245
-0.025028832537460903
0.0011822327509243885
0.016700422009116544
-0.0044798407167745665
-0.009666700000000005
0.02053182499265645
0.0037928760519934045
-0.010021373583397036
-0.15250578003670023
0.04601523399255922
-0.008985380343892926
-0.012655818146124202
0.011096

#### reference

* [Loc vs. iloc vs. ix vs. at vs. iat?](https://stackoverflow.com/questions/28757389/loc-vs-iloc-vs-ix-vs-at-vs-iat)

In [6]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

index = 1
mse = pd.Series(0.0, index=np.arange(2^5))
for f1 in [False, True]:
    for f2 in [False, True]:
        for f3 in [False, True]:
            for f4 in [False, True]:
                for f5 in [False, True]:
                    if not f1 and not f2 and not f3 and not f4 and not f5:
                        continue
                    select = [f1, f2, f3, f4, f5]
                    regr = linear_model.LinearRegression()
                    regr.fit(X.iloc[:, select], y)
                    diabetes_y_pred = regr.predict(X.iloc[:, select])
                    mse[index] = mean_squared_error(y, diabetes_y_pred)
                    print(index, select, mean_squared_error(y, diabetes_y_pred))
                    index += 1
                    
mse

1 [False, False, False, False, True] 0.04592205609241761
2 [False, False, False, True, False] 0.04598317898826777
3 [False, False, False, True, True] 0.04589030845834359
4 [False, False, True, False, False] 0.04598317898826777
5 [False, False, True, False, True] 0.04589030845834359
6 [False, False, True, True, False] 0.04598317898826777
7 [False, False, True, True, True] 0.04589030845834359
8 [False, True, False, False, False] 0.046008641930817815
9 [False, True, False, False, True] 0.04591590192781172
10 [False, True, False, True, False] 0.04597725806154627
11 [False, True, False, True, True] 0.04588429130959213
12 [False, True, True, False, False] 0.04597725806154627
13 [False, True, True, False, True] 0.04588429130959213
14 [False, True, True, True, False] 0.04598862734725435
15 [False, True, True, True, True] 0.04607599075547707
16 [True, False, False, False, False] 0.04599343968156881
17 [True, False, False, False, True] 0.04590171439631041
18 [True, False, False, True, False] 0.0

0     0.000000
1     0.045922
2     0.045983
3     0.045890
4     0.045983
5     0.045890
6     0.045983
7     0.045890
8     0.046009
9     0.045916
10    0.045977
11    0.045884
12    0.045977
13    0.045884
14    0.045989
15    0.046076
16    0.045993
17    0.045902
18    0.045963
19    0.045869
20    0.045963
21    0.045869
22    0.045964
23    0.046348
24    0.045981
25    0.045892
26    0.045953
27    0.045857
28    0.045953
29    0.045857
30    0.046248
31    0.046056
dtype: float64

In [7]:
y

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.179161
5       0.524784
6       0.657602
7       0.440675
8       0.587223
9       0.462075
10      0.644663
11      0.509006
12      0.538993
13      0.359575
14      0.537879
15      0.457736
16      0.475746
17      0.401232
18      0.607427
19      0.515183
20      0.701638
21      0.461361
22      0.447785
23      0.321535
24      0.548862
25      0.259916
26      0.408142
27      0.545569
28      0.601674
29      0.619231
          ...   
1230    0.323435
1231    0.450980
1232    0.688088
1233    0.287546
1234    0.390718
1235   -1.000000
1236    0.932529
1237    0.599379
1238    0.097171
1239    0.751523
1240    0.760979
1241    0.506091
1242    0.635464
1243    0.303274
1244    0.434130
1245    0.367387
1246    0.407798
1247    0.307571
1248    0.683339
1249    1.000000
1250    0.404448
1251    0.298011
1252    0.493846
1253    0.466133
1254    0.387242
1255    0.207858
1256    0.556325
1257    0.3927